In [1]:
from __future__ import annotations
import json, csv, sys, os, time
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import joblib
import pandas as pd
import numpy as np

# --- sklearn bits
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

# --- sentence-transformers for embeddings
try:
    from sentence_transformers import SentenceTransformer
except Exception as e:
    SentenceTransformer = None
    print("[WARN] sentence-transformers not importable right now. Install it to train/encode.", file=sys.stderr)

## Mapper (Part 1): Code

In [2]:
# ---------------- SBERT encoder (pipeline compatible) ----------------
class SBERTEncoder(BaseEstimator, TransformerMixin):
    """Lightweight sentence-embedding transformer for sklearn pipelines."""
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2", batch_size: int = 64, normalize: bool = True):
        self.model_name = model_name
        self.batch_size = batch_size
        self.normalize = normalize
        self._model = None

    def _ensure_model(self):
        if self._model is None:
            if SentenceTransformer is None:
                raise RuntimeError("sentence-transformers is required to encode prompts.")
            self._model = SentenceTransformer(self.model_name)

    def fit(self, X, y=None):
        self._ensure_model()
        return self

    def transform(self, X):
        self._ensure_model()
        embs = self._model.encode(
            list(X),
            batch_size=self.batch_size,
            show_progress_bar=False,
            normalize_embeddings=self.normalize,
        )
        return np.asarray(embs)

In [3]:
# ------------------------ I/O helpers ------------------------
def _read_prompts_jsonl(path: str) -> List[str]:
    prompts: List[str] = []
    with open(path, "r") as f:
        for line in f:
            try:
                rec = json.loads(line)
                p = rec.get("prompt", "").strip()
                if p:
                    prompts.append(p)
            except Exception:
                # tolerate occasional garbage lines
                continue
    return prompts

def _read_labels_csv(path: str) -> Dict[str, str]:
    gold: Dict[str, str] = {}
    df = pd.read_csv(path)
    if not {"prompt", "label"}.issubset(df.columns):
        raise ValueError(f"labels_csv must have columns ['prompt','label'], got {df.columns.tolist()}" )
    for _, row in df.iterrows():
        p = str(row["prompt"]).strip()
        y = str(row["label"]).strip()
        if p:
            gold[p] = y
    return gold

In [4]:
# ------------------------ Training ------------------------
def train_mapper(
    labels_csv: str,
    out_path: str = ".artifacts/defi_mapper.joblib",
    sbert_model: str = "sentence-transformers/all-MiniLM-L6-v2",
    C: float = 8.0,
    max_iter: int = 2000,
    calibrate: bool = True,
    calibration_method: str = "auto",  # 'auto' | 'isotonic' | 'sigmoid'
    calibration_cv: int = 3,
) -> str:
    """Train a SBERT + LogisticRegression pipeline, optionally calibrated, and dump to joblib."""
    os.makedirs(os.path.dirname(out_path) or ".", exist_ok=True)

    df = pd.read_csv(labels_csv)
    need = {"prompt","label"}
    if not need.issubset(df.columns):
        raise SystemExit(f"[train_mapper] labels_csv must have columns {need}, got {df.columns.tolist()}" )
    df = df.dropna(subset=["prompt","label"]).copy()
    df["prompt"] = df["prompt"].astype(str).str.strip()
    df["label"]  = df["label"].astype(str).str.strip()
    df = df[df["prompt"].str.len() > 0]
    if df.empty:
        raise SystemExit("[train_mapper] No non-empty prompts after cleaning.")

    X = df["prompt"].tolist()
    y = df["label"].tolist()

    base = LogisticRegression(max_iter=max_iter, C=C, class_weight="balanced", random_state=0)

    model = base
    if calibrate:
        # pick a safe calibration automatically for tiny classes
        from collections import Counter
        cnt = Counter(y); m = min(cnt.values())
        method = calibration_method; cv = calibration_cv
        if method == "auto":
            if m >= max(3, cv):
                method, cv = "isotonic", max(3, cv)
            elif m >= 2:
                method, cv = "sigmoid", max(2, min(m, cv))
            else:
                print("[train_mapper] Not enough per-class samples for calibration; skipping.", file=sys.stderr)
                method = None
        if method in ("isotonic","sigmoid"):
            try:
                model = CalibratedClassifierCV(estimator=base, method=method, cv=cv)  # sklearn>=1.3
            except TypeError:
                model = CalibratedClassifierCV(base_estimator=base, method=method, cv=cv)  # older sklearn

    pipe = make_pipeline(SBERTEncoder(sbert_model), model)
    pipe.fit(X, y)
    joblib.dump(pipe, out_path)
    print(f"[train_mapper] wrote: {out_path}  (n={len(X)})")
    return out_path

In [5]:
# ------------------------ Prediction & Metrics ------------------------
@dataclass
class PredictResult:
    rows_csv: Optional[str]
    metrics: Optional[dict]


def _predict_proba(mapper, prompts: List[str]) -> Tuple[List[str], np.ndarray]:
    if hasattr(mapper, "classes_"):
        classes = list(map(str, mapper.classes_))
    else:
        # try to infer from predict_proba later
        classes = None

    if hasattr(mapper, "predict_proba"):
        probs = mapper.predict_proba(prompts)
        probs = np.asarray(probs, dtype=float)
        if classes is None:
            classes = list(map(str, getattr(mapper, "classes_", [])))
        return classes, probs

    # decision_function -> softmax fallback
    if hasattr(mapper, "decision_function"):
        logits = mapper.decision_function(prompts)
        logits = np.asarray(logits, dtype=float)
        if logits.ndim == 1:
            logits = logits.reshape(-1, 1)
        ex = np.exp(logits - logits.max(axis=1, keepdims=True))
        probs = ex / (ex.sum(axis=1, keepdims=True) + 1e-12)
        if classes is None:
            classes = list(map(str, getattr(mapper, "classes_", [])))
        return classes, probs

    # predict-only fallback (degenerate probs)
    preds = np.array(mapper.predict(prompts), dtype=object).reshape(-1, 1)
    classes = list(sorted(set(map(str, preds.flatten().tolist()))))
    idx = {c:i for i,c in enumerate(classes)}
    probs = np.zeros((len(prompts), len(classes)), dtype=float)
    for r, y in enumerate(preds.flatten().tolist()):
        probs[r, idx[str(y)]] = 1.0
    return classes, probs


def predict_prompts(
    mapper_path: str,
    prompts_jsonl: str,
    labels_csv_pred: Optional[str] = None,
    threshold: float = 0.6,
    out_rows_csv: Optional[str] = None,
) -> PredictResult:
    """Load mapper, score prompts, write per-row CSV, and (optionally) compute quick metrics."""
    mapper = joblib.load(mapper_path)
    prompts = _read_prompts_jsonl(prompts_jsonl)
    classes, probs = _predict_proba(mapper, prompts)

    top_idx = probs.argmax(axis=1)
    top_conf = probs.max(axis=1)
    fired = (top_conf >= threshold)
    preds = np.array([classes[i] for i in top_idx], dtype=object)
    pred_labels = np.where(fired, preds, "")

    rows = []
    if labels_csv_pred:
        gold = _read_labels_csv(labels_csv_pred)
    else:
        gold = {}

    for p, yhat, conf, fire in zip(prompts, pred_labels, top_conf, fired):
        rows.append({
            "prompt": p,
            "predicted": yhat,
            "confidence": float(conf),
            "abstain": (not bool(fire)),
            "threshold": float(threshold),
            "gold_label": gold.get(p, "")
        })

    rows_csv_path = None
    if out_rows_csv:
        os.makedirs(os.path.dirname(out_rows_csv) or ".", exist_ok=True)
        import csv as _csv
        with open(out_rows_csv, "w", newline="") as f:
            w = _csv.DictWriter(f, fieldnames=["prompt","gold_label","predicted","confidence","abstain","threshold"]
            )
            w.writeheader()
            for r in rows:
                w.writerow({k: r[k] for k in w.fieldnames})
        rows_csv_path = out_rows_csv
        print(f"[predict] wrote rows: {rows_csv_path}  (n={len(rows)})")

    # quick metrics (if gold labels provided)
    metrics = None
    if gold:
        total = len(prompts)
        abstain_ct = sum(1 for r in rows if r["abstain"])
        fired_ct   = total - abstain_ct
        correct_on_fired = sum(1 for r in rows if (not r["abstain"]) and r["predicted"] == r["gold_label"])
        overall_correct  = sum(1 for r in rows if r["predicted"] == r["gold_label"])  # empty pred never equals gold

        accuracy_on_fired = (correct_on_fired / fired_ct) if fired_ct else None
        overall_accuracy  = overall_correct / total if total else None

        metrics = {
            "threshold": float(threshold),
            "total": total,
            "abstain": abstain_ct,
            "abstain_rate": abstain_ct / total if total else None,
            "coverage": fired_ct / total if total else None,
            "fired": fired_ct,
            "correct_on_fired": correct_on_fired,
            "accuracy_on_fired": accuracy_on_fired,
            "overall_correct": overall_correct,
            "overall_accuracy": overall_accuracy,
        }
        print("[metrics]", metrics)

    return PredictResult(rows_csv=rows_csv_path, metrics=metrics)

# ------------------------ CLI (optional) ------------------------
def _as_bool(x: str) -> bool:
    return str(x).strip().lower() in {"1","true","t","yes","y"}

In [6]:
import argparse, csv, json, os, sys, time
def get_args():
    ap = argparse.ArgumentParser()
    ap.add_argument("--backend",       default="wordmap", help="wordmap|sbert")
    ap.add_argument("--model_path",    default=".artifacts/defi_mapper.joblib")
    ap.add_argument("--prompts_jsonl", default="tests/fixtures/defi/defi_mapper_5k_prompts.json")
    ap.add_argument("--labels_csv_pred", default="tests/fixtures/defi/defi_mapper_labeled_5k.csv")
    ap.add_argument("--train_labels_csv", default="tests/fixtures/defi/defi_mapper_labeled_large.csv")
    ap.add_argument("--thresholds",    default="0.5,0.55,0.6,0.65,0.7")
    ap.add_argument("--max_iter",    default="2000")
    ap.add_argument("--C",    default="8")
    ap.add_argument("--calibrate",    default="True")
    ap.add_argument("--calibration_method", choices=["auto","isotonic","sigmoid"], default="auto")
    ap.add_argument("--calibration_cv", type=int, default=3)
    ap.add_argument("--sbert_model", default="sentence-transformers/all-MiniLM-L6-v2")
    ap.add_argument("--threshold", type=float, default=0.6)
    ap.add_argument("--out_path", default="defi_mapper_embed.joblib")
    ap.add_argument("--out_dir",       default="")
    ap.add_argument("--out_rows_csv", default=".artifacts/m8_rows_simple.csv")
    ap.add_argument("--min_overall_acc", default=None)
    
    notebook_args = [
        "--backend", "sbert",
        "--model_path", ".artifacts/defi_mapper.joblib",
        "--prompts_jsonl", "tests/fixtures/defi/defi_mapper_5k_prompts.jsonl",
        "--labels_csv_pred",    "tests/fixtures/defi/defi_mapper_labeled_5k.csv",
        "--train_labels_csv", "tests/fixtures/defi/defi_mapper_labeled_large.csv",
        "--thresholds", "0.5,0.55,0.6,0.65,.7",
        "--max_iter", "2000",
        "--C", "8",
        "--calibrate", "True",
        "--calibration_method", "auto",
        "--calibration_cv", "3",
        "--threshold", "0.5",
        "--min_overall_acc", "0.75",
        "--sbert_model", "sentence-transformers/all-MiniLM-L6-v2",
        "--out_path", "defi_mapper_embed.joblib",
        "--out_rows_csv", ".artifacts/m8_rows_simple.csv",
        "--out_dir", ".artifacts/defi/mapper_bench",
    ]
    
    return ap.parse_args(notebook_args)

## Mapper (Part 2): Train/Test

In [7]:
import os
cwd =  os.getcwd().replace("/notebooks","")
os.chdir(cwd)

args = get_args()

model_path = train_mapper(
        labels_csv=args.train_labels_csv,
        out_path=args.out_path,
        sbert_model=args.sbert_model,
        C=float(args.C),
        max_iter=int(args.max_iter),
        calibrate=_as_bool(args.calibrate),
        calibration_method=args.calibration_method,
        calibration_cv=args.calibration_cv
    )


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[train_mapper] wrote: defi_mapper_embed.joblib  (n=1000)


In [8]:
args = get_args()

predict_prompts(
        mapper_path=model_path,
        prompts_jsonl=args.prompts_jsonl,
        labels_csv_pred=(args.labels_csv_pred or None),
        threshold=float(args.threshold),
        out_rows_csv=args.out_rows_csv
    )

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[predict] wrote rows: .artifacts/m8_rows_simple.csv  (n=5000)
[metrics] {'threshold': 0.5, 'total': 5000, 'abstain': 37, 'abstain_rate': 0.0074, 'coverage': 0.9926, 'fired': 4963, 'correct_on_fired': 4941, 'accuracy_on_fired': 0.995567197259722, 'overall_correct': 4941, 'overall_accuracy': 0.9882}


PredictResult(rows_csv='.artifacts/m8_rows_simple.csv', metrics={'threshold': 0.5, 'total': 5000, 'abstain': 37, 'abstain_rate': 0.0074, 'coverage': 0.9926, 'fired': 4963, 'correct_on_fired': 4941, 'accuracy_on_fired': 0.995567197259722, 'overall_correct': 4941, 'overall_accuracy': 0.9882})

In [9]:
# mapper
mapper = joblib.load(model_path)
test_prompts = _read_prompts_jsonl(args.prompts_jsonl)
labeled_test_prompts = _read_labels_csv(args.labels_csv_pred)
threshold = float(args.threshold)

classes, probs = _predict_proba(mapper, test_prompts[0:5])

top_idx = probs.argmax(axis=1)
top_conf = probs.max(axis=1)
fired = (top_conf >= threshold)
preds = np.array([classes[i] for i in top_idx], dtype=object)
pred_labels = np.where(fired, preds, "")
pred_labels

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


array(['claim_rewards', 'withdraw_asset', 'stake_asset', 'deposit_asset',
       'deposit_asset'], dtype=object)

In [10]:
test_prompts = ['sing me a lullaby',
                      'what’s the weather in NYC?',
                      'open settings',
                      'convert centimeters to inches',
                      'swap seats with me',
                      'hey wats going on']

# mapper
mapper = joblib.load(model_path)
test_prompts = _read_prompts_jsonl(args.prompts_jsonl)
labeled_test_prompts = _read_labels_csv(args.labels_csv_pred)
threshold = float(args.threshold)

classes, probs = _predict_proba(mapper, test_prompts[0:5])

top_idx = probs.argmax(axis=1)
top_conf = probs.max(axis=1)
fired = (top_conf >= threshold)
preds = np.array([classes[i] for i in top_idx], dtype=object)
pred_labels = np.where(fired, preds, "")
pred_labels

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


array(['claim_rewards', 'withdraw_asset', 'stake_asset', 'deposit_asset',
       'deposit_asset'], dtype=object)

In [11]:
# mapper
mapper = joblib.load(model_path)
test_prompts = _read_prompts_jsonl(args.prompts_jsonl)
labeled_test_prompts = _read_labels_csv(args.labels_csv_pred)
threshold = float(args.threshold)

classes, probs = _predict_proba(mapper, test_prompts[0:5])

top_idx = probs.argmax(axis=1)
top_conf = probs.max(axis=1)
fired = (top_conf >= threshold)
preds = np.array([classes[i] for i in top_idx], dtype=object)
pred_labels = np.where(fired, preds, "")
pred_labels

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


array(['claim_rewards', 'withdraw_asset', 'stake_asset', 'deposit_asset',
       'deposit_asset'], dtype=object)

## Auditor (Part 2): Setup SbertEmbedding

In [12]:
PRIMITIVE = "swap_asset"

# get training prompts
N_TRAIN = 1000
train_prompts = _read_labels_csv(args.train_labels_csv)
train_deposit_labels = []
labeled_train_deposit_prompts = {}

for k, g in enumerate(train_prompts):

    if(train_prompts[g] == PRIMITIVE):
        labeled_train_deposit_prompts[k] = g
        train_deposit_labels.append(1)
    else:
        train_deposit_labels.append(0)
        
    if (k == N_TRAIN):
        break;

train_deposit_prompts = list(labeled_train_deposit_prompts.values())

In [13]:
# get test prompts
N_TEST = 25
labeled_test_prompts = _read_labels_csv(args.labels_csv_pred)
labeled_test_deposit_prompts = {}
for k, g in enumerate(labeled_test_prompts):

    if(labeled_test_prompts[g] == PRIMITIVE):
        labeled_test_deposit_prompts[k] = g
    if (len(labeled_test_deposit_prompts) == N_TEST):
        break;

test_deposit_seeds = list(labeled_test_prompts.keys())
test_seeds = list(labeled_test_deposit_prompts.values())
labeled_test_deposit_prompts

{5: 'convert 7064 USDT into LINK via sushiswap on arbitrum — minimize gas, safe mode',
 20: 'convert 704.7582 AAVE into USDT via uniswap on arbitrum — minimize gas',
 23: 'swap 860.4607 ARB to WBTC on sushiswap arbitrum with slippage 0.1% — asap',
 31: 'swap 0.240081 WBTC to WETH on balancer solana',
 33: 'market swap 5930 USDT->SOL using sushiswap on arbitrum — safe mode',
 42: 'market swap 26.3876 MATIC->WBTC using sushiswap on polygon with slippage 0.1% — right away',
 46: 'trade 6823 USDC for ETH on curve (arbitrum) — now',
 60: 'trade 3389.1714 OP for WBTC on curve (base) with slippage 1% — asap',
 61: 'swap 3229.3468 LINK to AAVE on curve avalanche with slippage 0.1%',
 63: 'trade 32.883 MATIC for WBTC on curve (avalanche)',
 64: 'convert 1216 USDT into WETH via balancer on ethereum',
 70: 'swap 684.2186 LINK to USDC on uniswap arbitrum with slippage 0.2% — minimize gas',
 78: 'market swap 25.7623 ETH->SOL using balancer on avalanche with slippage 1%',
 79: 'swap 22.1568 SOL to A

In [14]:
try:
    from sentence_transformers import SentenceTransformer
    _SBERT_OK = True
except Exception:
    _SBERT_OK = False

class SbertEmbedding:
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        if not _SBERT_OK:
            raise RuntimeError("sentence-transformers not available; pip install sentence-transformers")
        self.m = SentenceTransformer(model_name)
    def encode_one(self, text: str) -> np.ndarray:
        v = self.m.encode([text], normalize_embeddings=True)[0]
        return v.astype(np.float32)

def load_phrases():
    default = {
        "deposit_asset": ["deposit","top up","add","add funds","fund","supply","put in", "move","provide"],
        "withdraw_asset": ["withdraw","cash out","take out","remove","pull"],
        "swap_asset": ["swap","convert","exchange","trade"],
        "borrow_asset": ["borrow","take loan","obtain credit","draw"],
        "repay_asset": ["repay","pay back","settle debt","return loan"],
        "stake_asset": ["stake","lock","bond","delegate"],
        "unstake_asset": ["unstake","unlock","withdraw stake","redeem stake"],
        "claim_rewards": ["claim","collect","harvest","receive rewards"]
    }
    return default

def build_prototypes(emb, phrases_or_thresholds: dict):
    """
    Accepts either:
      A) phrases: {prim: [(phrase, score), (phrase, score), ...]}  or  {prim: [phrase, phrase, ...]}
      B) thresholds: {prim: float}  (per-class threshold file)
    For (B), we fall back to embedding the primitive name itself.
    """
    proto = {}
    for p, val in phrases_or_thresholds.items():
        vecs = []
        if isinstance(val, (int, float)):
            # thresholds file -> fallback: use primitive token as prototype
            vecs = [emb.encode_one(p)]
        else:
            # phrases file: list of phrases or (phrase,score)
            if isinstance(val, list) and len(val) > 0:
                for item in val:
                    if isinstance(item, (list, tuple)) and len(item) > 0:
                        phrase = item[0]
                    else:
                        phrase = item
                    vecs.append(emb.encode_one(str(phrase)))
            else:
                # empty -> fallback
                vecs = [emb.encode_one(p)]
        proto[p] = np.stack(vecs, axis=0).mean(axis=0)
    return proto
    
def cosine(a, b):
    an = a / (np.linalg.norm(a) + 1e-8)
    bn = b / (np.linalg.norm(b) + 1e-8)
    return float(np.dot(an, bn))

def spans_from_prompt(prompt, prototypes, emb, tau_span=0.55):
    toks = prompt.strip().split()
    spans = []
    for n in range(1, min(6, len(toks))+1):
        for i in range(0, len(toks)-n+1):
            s = " ".join(toks[i:i+n])
            e = emb.encode_one(s)
            for k, v in prototypes.items():
                sc = max(0.0, cosine(e, v))
                if sc >= tau_span:
                    t_center = (i + n/2.0) / max(1.0, len(toks))  # normalized [0,1]
                    spans.append({"primitive": k, "term": s, "score": round(sc,4), "t_center": round(t_center,4)})
    # keep top 3 spans per primitive
    by_prim = {}
    for sp in spans:
        by_prim.setdefault(sp["primitive"], []).append(sp)
    for k in by_prim:
        by_prim[k] = sorted(by_prim[k], key=lambda x: x["score"], reverse=True)[:3]
    return by_prim

span_map = spans_from_prompt(test_prompt, prototypes, emb, tau_span=0.55)
span_map

In [15]:
emb = SbertEmbedding()
phrases = load_phrases()
prototypes = build_prototypes(emb, phrases)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [22]:
test_prompt = test_seeds[14]
tau_span = 0.55

# 1) primitive mapping (whole-sentence sim)
e = emb.encode_one(test_prompt)
primitive_mapping = {k: round(max(0.0, cosine(e, v)), 4) for k, v in prototypes.items()}
primitive_mapping

{'deposit_asset': 0.0837,
 'withdraw_asset': 0.0531,
 'swap_asset': 0.1479,
 'borrow_asset': 0.0,
 'repay_asset': 0.0251,
 'stake_asset': 0.0,
 'unstake_asset': 0.0898,
 'claim_rewards': 0.0}

In [23]:
# 2) primitive -> term mapping (span detection)
span_map = spans_from_prompt(test_prompt, prototypes, emb, tau_span=0.55)
span_map

{'swap_asset': [{'primitive': 'swap_asset',
   'term': 'convert',
   'score': 0.6612,
   't_center': 0.0417}],
 'deposit_asset': [{'primitive': 'deposit_asset',
   'term': 'into',
   'score': 0.5735,
   't_center': 0.2917}]}

## Auditor (Part 2): Audit from SBERT span map

In [18]:
negative_test_seeds = ['sing me a lullaby',
                      'what’s the weather in NYC?',
                      'open settings',
                      'convert centimeters to inches',
                      'swap seats with me',
                      'hey wats going on']

In [19]:
import json, math, hashlib
import numpy as np

# ---------- utilities ----------
def kaiser_window(L=160, beta=8.6):
    # smooth, MF-friendly lobe (unit norm)
    n = np.arange(L)
    w = np.i0(beta * np.sqrt(1 - ((2*n)/(L-1) - 1)**2))
    w = w / (np.linalg.norm(w) + 1e-9)
    return w.astype("float32")

def matched_filter_scores(traces, q):
    scores, nulls, peaks = {}, {}, {}
    L = len(q)
    for k, x in traces.items():
        if len(x) < L:  # pad if needed
            x = np.pad(x, (0, L - len(x)))
        # convolution as correlation (flip q)
        r = np.convolve(x, q[::-1], mode="valid")
        peak = float(r.max()) if r.size else 0.0
        scores[k] = peak
        nulls[k]  = float(np.sqrt(np.sum(x**2)) * (np.linalg.norm(q))) / max(len(x),1)  # crude noise floor
        peaks[k]  = {"score": peak, "t_idx": int(np.argmax(r)) if r.size else 0}
    return scores, nulls, peaks

def decide(scores, nulls, tau_rel=0.60, tau_abs=0.93):
    accepted, seq = {}, []
    for k in scores:
        s, n = scores[k], nulls[k] + 1e-9
        rel = s / n
        if (rel >= tau_rel) and (s >= tau_abs):
            accepted[k] = {"score": round(s, 4), "rel": round(rel, 3), "null": round(n, 4)}
            seq.append((k, s))
    seq.sort(key=lambda z: -z[1])
    return [k for k, _ in seq], accepted

# ---------- core: audit from spans ----------
def audit_from_span_map(prompt:str,
                        primitive_to_term_mapping:dict,
                        T:int=720,
                        tau_span:float=0.55,
                        tau_abs:float=0.93,
                        tau_rel:float=0.60,
                        sigma:float=0.02,
                        fuse_per_primitive:bool=False):
    """
    primitive_to_term_mapping: {
      "deposit_asset": [{"term":"add","score":0.6391,"t_center":0.0833}, ...],
      ...
    }
    """
    # 1) init noise traces
    seed = int(hashlib.sha256(prompt.encode("utf-8")).hexdigest()[:8], 16)
    rng  = np.random.default_rng(seed)
    primitives = list(primitive_to_term_mapping.keys())
    traces = {p: rng.normal(0.0, sigma, size=T).astype("float32") for p in primitives}

    # 2) build a canonical lobe shape
    q = kaiser_window(L=min(160, max(64, T//8)))

    # 3) inject lobes from strong spans
    span_hits = 0
    for p, hits in primitive_to_term_mapping.items():
        if not hits: 
            continue
        # keep only strong spans
        strong = [h for h in hits if float(h.get("score",0.0)) >= tau_span]
        if not strong:
            continue

        if fuse_per_primitive:
            # one fused lobe per primitive (amplitude = max span score)
            A = max(float(h["score"]) for h in strong)
            tc = np.mean([float(h.get("t_center", 0.5)) for h in strong])
            start = max(0, min(T - len(q), int(tc * (T - len(q)))))
            traces[p][start:start+len(q)] += A * q
            span_hits += 1
        else:
            # one lobe per span
            for h in strong:
                A  = float(h["score"])
                tc = float(h.get("t_center", 0.5))
                start = max(0, min(T - len(q), int(tc * (T - len(q)))))
                traces[p][start:start+len(q)] += A * q
                span_hits += 1

    if span_hits == 0:
        return {
            "decision": "ABSTAIN",
            "sequence": [],
            "accepted_peaks": {},
            "notes": {"reason": "no_span_evidence", "tau_span": tau_span, "T": T}
        }

    # 4) matched filter + parser
    scores, nulls, peaks = matched_filter_scores(traces, q)
    sequence, accepted = decide(scores, nulls, tau_rel=tau_rel, tau_abs=tau_abs)

    return {
        "decision": "PASS" if sequence else "ABSTAIN",
        "sequence": sequence,
        "accepted_peaks": accepted,
        "peaks": peaks,  # optional: raw peak info
        "notes": {
            "tau_span": tau_span, "tau_rel": tau_rel, "tau_abs": tau_abs,
            "sigma": sigma, "T": T, "fused": fuse_per_primitive
        }
    }

# ---------- demo ----------
if __name__ == "__main__":

    for k, test_prompt in enumerate(negative_test_seeds):
        span_map = spans_from_prompt(test_prompt, prototypes, emb, tau_span=tau_span)
        if(PRIMITIVE in span_map):    
            primitive_to_term_mapping = {
                "deposit_asset":  [],       
                "withdraw_asset": [],
                "swap_asset": [],
                "borrow_asset": [],
                "repay_asset": [],
                "stake_asset": [],
                "unstake_asset": [],
                "claim_rewards": []
            }

            primitive_to_term_mapping[PRIMITIVE] = span_map[PRIMITIVE]
            
            audit = audit_from_span_map(
                test_prompt,
                primitive_to_term_mapping,
                T=720, tau_span=0.55, tau_abs=0.50, tau_rel=0.60, sigma=0.02,
                fuse_per_primitive=False
            )
            #print(json.dumps(audit, indent=2))
            is_passed = audit['decision']
            print(f'{k} {is_passed} / prompt: {test_prompt} / primitives: {list(span_map.keys())}')
        else:
            print(f'{k} ABSTAIN / prompt: {test_prompt}')
    
        

0 ABSTAIN / prompt: sing me a lullaby
1 ABSTAIN / prompt: what’s the weather in NYC?
2 ABSTAIN / prompt: open settings
3 PASS / prompt: convert centimeters to inches / primitives: ['swap_asset']
4 PASS / prompt: swap seats with me / primitives: ['swap_asset']
5 ABSTAIN / prompt: hey wats going on


In [20]:
test_prompts = ['sing me a lullaby',
                      'what’s the weather in NYC?',
                      'open settings',
                      'convert centimeters to inches',
                      'swap seats with me',
                      'hey wats going on']

# mapper
mapper = joblib.load(model_path)
# test_prompts = _read_prompts_jsonl(args.prompts_jsonl)
labeled_test_prompts = _read_labels_csv(args.labels_csv_pred)
threshold = float(args.threshold)

classes, probs = _predict_proba(mapper, test_prompts)

top_idx = probs.argmax(axis=1)
top_conf = probs.max(axis=1)
fired = (top_conf >= threshold)
preds = np.array([classes[i] for i in top_idx], dtype=object)
pred_labels = np.where(fired, preds, "")
pred_labels

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


array(['claim_rewards', 'deposit_asset', 'deposit_asset',
       'withdraw_asset', '', ''], dtype=object)

## ChatGPT Fix

### 0) seed terms + encoder + prototypes

In [62]:
# ===== Cell 0: seed term bank (tight, unambiguous) =====
TERM_BANK = {
    "deposit_asset":  ["deposit", "supply", "provide"],
    "withdraw_asset": ["withdraw", "redeem", "unstow"],
    "swap_asset":     ["swap", "convert", "trade", "exchange"],
    "borrow_asset":   ["borrow", "draw"],
    "repay_asset":    ["repay", "pay back"],
    "stake_asset":    ["stake", "lock", "bond"],
    "unstake_asset":  ["unstake", "unlock", "unbond"],
    "claim_rewards":  ["claim", "harvest", "collect rewards"],
}
PRIMS = list(TERM_BANK.keys())


In [63]:
# ===== Cell 1: SBERT encoder wrapper =====
from sentence_transformers import SentenceTransformer
import numpy as np

class Emb:
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        self._m = SentenceTransformer(model_name)
    def encode(self, texts, normalize_embeddings=True, show_progress_bar=False):
        return self._m.encode(
            texts, normalize_embeddings=normalize_embeddings,
            show_progress_bar=show_progress_bar
        )
    def encode_one(self, text, normalize_embeddings=True):
        return self.encode([text], normalize_embeddings=normalize_embeddings)[0]

emb = Emb()  # or Emb("sentence-transformers/all-mpnet-base-v2")


In [64]:
# ===== Cell 2: build primitive -> prototype vector (centroid over seed terms) =====
def build_prototypes(term_bank: dict[str, list[str]], emb: Emb) -> dict[str, np.ndarray]:
    protos = {}
    for prim, terms in term_bank.items():
        V = np.asarray(emb.encode(terms, normalize_embeddings=True))
        protos[prim] = V.mean(axis=0)
    return protos

prototypes = build_prototypes(TERM_BANK, emb)


## 1) spans_from_prompt (your style, dict-only return)

In [65]:
# ===== Cell 3: spans_from_prompt (dict-only return) =====
import re

_WORD = re.compile(r"[a-z0-9]+(?:'[a-z0-9]+)?")

def _norm_tokens(txt: str) -> list[str]:
    txt = txt.lower()
    return _WORD.findall(txt)

def _cosine(a: np.ndarray, b: np.ndarray) -> float:
    da = float(np.linalg.norm(a) + 1e-9)
    db = float(np.linalg.norm(b) + 1e-9)
    return float(np.dot(a, b) / (da * db))

def spans_from_prompt(prompt: str,
                      prototypes: dict[str, np.ndarray],
                      emb: Emb,
                      tau_span: float = 0.55,
                      n_max: int = 5,
                      topk_per_prim: int = 3):
    """
    Returns: dict primitive -> [ {primitive, term, score, t_center, start, len}, ... ] (top-k per primitive)
    """
    toks = _norm_tokens(prompt)
    if not toks:
        return {}

    grams, meta = [], []   # meta holds (start, n, t_center)
    for n in range(1, min(n_max, len(toks)) + 1):
        for i in range(0, len(toks) - n + 1):
            s = " ".join(toks[i:i+n])
            t_center = (i + n/2.0) / max(1.0, len(toks))
            grams.append(s)
            meta.append((i, n, t_center))

    V = emb.encode(grams, normalize_embeddings=True, show_progress_bar=False)  # [M, D]

    by_prim: dict[str, list[dict]] = {k: [] for k in prototypes.keys()}
    for m, (i, n, t_center) in enumerate(meta):
        v = V[m]
        for prim, proto in prototypes.items():
            sc = max(0.0, _cosine(v, proto))
            if sc >= tau_span:
                by_prim[prim].append({
                    "primitive": prim,
                    "term": grams[m],
                    "score": round(sc, 4),
                    "t_center": round(t_center, 4),
                    "start": i,
                    "len": n,
                })

    # keep top-k per primitive by score
    for prim in list(by_prim.keys()):
        arr = sorted(by_prim[prim], key=lambda x: x["score"], reverse=True)[:topk_per_prim]
        if arr:
            by_prim[prim] = arr
        else:
            # drop empty lists to make downstream checks easy
            by_prim.pop(prim, None)

    return by_prim


### 2) audit wrapper (robust to dict-only or (dict, meta) variants)

In [66]:
# ===== Cell 4: audit_prompt_with_spans (robust wrapper) =====
def audit_prompt_with_spans(prompt: str,
                            prototypes: dict[str, np.ndarray],
                            emb: Emb,
                            tau_span: float = 0.55,
                            rel_margin: float = 0.06):
    """
    Uses spans_from_prompt to produce:
      - best_primitive: lexical 'winner' (requires tau + small relative margin)
      - scores: top-score per primitive (0 if none)
      - spans: raw span map from spans_from_prompt
      - rel_margin: best - second best score
    Works whether spans_from_prompt returns dict or (dict, meta).
    """
    out = spans_from_prompt(prompt, prototypes, emb, tau_span=tau_span)
    if isinstance(out, tuple):
        span_map, meta = out
    else:
        span_map, meta = out, {}

    # top score per primitive (ensure we cover all prototypes, even if absent in span_map)
    scores = {}
    for prim in prototypes.keys():
        lst = span_map.get(prim, [])
        scores[prim] = (lst[0]["score"] if lst else 0.0)

    ordered = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)
    best_prim, best_sc = ordered[0]
    second_sc = ordered[1][1] if len(ordered) > 1 else 0.0
    winner = best_prim if (best_sc >= tau_span and (best_sc - second_sc) >= rel_margin) else None

    return {
        "best_primitive": winner,
        "scores": scores,
        "spans": span_map,
        "rel_margin": best_sc - second_sc,
        "meta": meta,
        "params": {"tau_span": tau_span, "rel_margin": rel_margin},
    }


### 3) opposite-primitive veto (kills tautologies)

In [67]:
# ===== Cell 5: opposite/veto helpers =====
OPPOSITE = {
    "deposit_asset": "withdraw_asset",
    "withdraw_asset": "deposit_asset",
    "stake_asset": "unstake_asset",
    "unstake_asset": "stake_asset",
    "borrow_asset": "repay_asset",
    "repay_asset": "borrow_asset",
    # claim_rewards has no strict opposite in this simple map
}

def should_veto(mapper_top1: str | None, audit_best: str | None) -> bool:
    if not mapper_top1 or not audit_best:
        return False
    return OPPOSITE.get(mapper_top1) == audit_best


### 4) fuse with mapper (single call to get final decision)

In [68]:
# ===== Cell 6: fuse_decision (mapper + spans audit) =====
import joblib

def load_mapper(path=".artifacts/defi_mapper.joblib"):
    return joblib.load(path)

def mapper_top1_label(mapper, prompt: str):
    # generic scikit-like pipeline
    if hasattr(mapper, "predict_proba"):
        probs = mapper.predict_proba([prompt])[0]
        classes = list(getattr(mapper, "classes_", PRIMS))
        top_idx = int(np.argmax(probs))
        return classes[top_idx], float(probs[top_idx])
    else:
        lab = mapper.predict([prompt])[0]
        return str(lab), 1.0

def fuse_decision(prompt: str,
                  mapper,
                  prototypes: dict[str, np.ndarray],
                  emb: Emb,
                  conf_thr: float = 0.70,
                  tau_span: float = 0.55,
                  rel_margin: float = 0.06):
    m_top, m_conf = mapper_top1_label(mapper, prompt)
    fired = bool(m_conf >= conf_thr)

    audit = audit_prompt_with_spans(prompt, prototypes, emb, tau_span=tau_span, rel_margin=rel_margin)
    a_best = audit["best_primitive"]

    if not fired:
        return {
            "prompt": prompt,
            "decision": "abstain_non_exec",
            "reason": "low_conf_mapper",
            "mapper": {"top": m_top, "conf": m_conf},
            "audit": audit,
        }

    if should_veto(m_top, a_best):
        return {
            "prompt": prompt,
            "decision": "reject",
            "reason": f"tautology_veto:{m_top}_vs_{a_best}",
            "mapper": {"top": m_top, "conf": m_conf},
            "audit": audit,
        }

    # optional: require lexical alignment when audit is strong
    if a_best and a_best != m_top:
        return {
            "prompt": prompt,
            "decision": "reject",
            "reason": f"audit_mismatch:{m_top}_vs_{a_best}",
            "mapper": {"top": m_top, "conf": m_conf},
            "audit": audit,
        }

    return {
        "prompt": prompt,
        "decision": "approve",
        "mapper": {"top": m_top, "conf": m_conf},
        "audit": audit,
    }


### 5) quick smoke (copy/paste)

In [73]:
# ===== Cell 7: quick smoke =====
PRIMS = list(prototypes.keys())
mapper = joblib.load(model_path)
tests = [
    "deposit 10 ETH into aave",
    "withdraw 5 ETH",
    "swap 2 ETH for USDC on uniswap — minimize gas",
    "check balance",
    "repay 300 USDC to aave",
    "unstake 1000 USDC from rocket pool",
    "provide 4569.792 OP to balancer on arbitrum — asap",
    "swap 991.2209 WETH to MATIC on compound",
    "unstake and take out 1941.4165 LINK — use normal gas",
    "supply 231.3364 USDC — safe mode",
    "withdraw 2728.1894 ARB — asap",
    "open a loan for 1495.415 SOL — asap",
    "sing me a lullaby",
    "convert centimeters to inches"
]
for p in tests:
    out = fuse_decision(p, mapper, prototypes, emb, conf_thr=0.70, tau_span=0.55, rel_margin=0.06)
    print(out["decision"], "—", p)
    if out["decision"] != "approve":
        print("  reason:", out.get("reason"))


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is d

approve — deposit 10 ETH into aave
approve — withdraw 5 ETH
approve — swap 2 ETH for USDC on uniswap — minimize gas
abstain_non_exec — check balance
  reason: low_conf_mapper
approve — repay 300 USDC to aave
approve — unstake 1000 USDC from rocket pool


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


approve — provide 4569.792 OP to balancer on arbitrum — asap
abstain_non_exec — swap 991.2209 WETH to MATIC on compound
  reason: low_conf_mapper


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


abstain_non_exec — unstake and take out 1941.4165 LINK — use normal gas
  reason: low_conf_mapper
abstain_non_exec — supply 231.3364 USDC — safe mode
  reason: low_conf_mapper
abstain_non_exec — withdraw 2728.1894 ARB — asap
  reason: low_conf_mapper
approve — open a loan for 1495.415 SOL — asap
abstain_non_exec — sing me a lullaby
  reason: low_conf_mapper
reject — convert centimeters to inches
  reason: audit_mismatch:withdraw_asset_vs_swap_asset


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/modules/module.py:1784: FutureWarning: `encoder_attention_mask` is d